# Horarios de reloj formateados

importar librerias

In [182]:
import pandas as pd
import os
import datetime

obtener ruta de archivos

In [183]:
base_dir = os.path.dirname(os.path.realpath('__file__'))
descargas = os.path.join(base_dir, 'descargas')
listado = os.listdir(descargas)
output = os.path.join(base_dir, 'output')
existe = os.path.exists(output)
if not existe:
    os.makedirs(output)    

iterar

In [208]:

#crear dict, encabezado y asignar una lista vacia
main_dict = {}
cols = ['fecha', 'entrada', 'salida', 'hs_trab', 'observaciones' , 'nombre']
for i in range(len(cols)):
            main_dict[cols[i]] = []
        
#iterar los archivos
for l in listado:
    files = os.path.join(descargas, l)
    data_s = pd.read_excel(files)
    
    nombre = data_s['Nombre'][1]    
    col_fh = data_s['Fecha/Hora']
    
    temp = {}

    
    
    #contar si marca 1 o 2 veces
    cont = {}
    for r in col_fh:
        fecha, _ = str(r).split(' ')
        if fecha in cont:
            cont[fecha] = 2
        else:
            cont[fecha] = 1
    
    #si marco una vez agregar otra marca
    for i, j in cont.items():    
        if j == 1:     
            #i += ' --'
            dt_string = f'{i} 00:00:00'
            i = datetime.datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S")
            #format = '%I:%M' # The format
            #datetime_str = datetime.datetime.strptime(i, format)
            
            s1 = pd.Series(i)            
            col_fh =  col_fh.append(s1)
            
    
            
    col_fh = col_fh.sort_values()
    
    #agregar salto de linea
    fila = [ '', '', '' , '', '', '']
    fila1 = [ 'Nombre', f'{nombre}', '' , '', '', '']
    for i in range(len(cols)):
        main_dict[cols[i]].append(fila[i])
        main_dict[cols[i]].append(fila1[i]) 
        main_dict[cols[i]].append(cols[i])

    #recorrer la columna fecha-hora
    for row in col_fh:
        fecha, hora = str(row).split(' ')
        try:
            #print(hora)
            if hora == '00:00:00':
                hora = 'None'
            else:               
                hora = datetime.datetime.strptime(hora, '%H:%M:%S')
            
         
                
        except:
            hora = 'None'
            
                
        if fecha not in temp:
            temp[fecha] = []
            temp[fecha].append(fecha)
            temp[fecha].append(hora)
        else:
            temp[fecha].append(hora)
            fecha = temp[fecha][0]
            entrada = str(temp[fecha][1])[11:16]
            salida = str(temp[fecha][2])[11:16]              
            try:
                horas_t = str(temp[fecha][2] - temp[fecha][1])[:4]
            except:
                horas_t = 'None'     
            obs= '-' 

            fila = [fecha, entrada, salida , horas_t, obs, nombre]

            for i in range(len(cols)):
                main_dict[cols[i]].append(fila[i]) 
            

       
           
df = pd.DataFrame(main_dict)

#df = df.sort_values(by='fecha')

print(df)
#df.to_excel(f"{output}\{filename}.xlsx", sheet_name='reporte', 
#                columns=['fecha', 'entrada' ,'salida','hs_trab','observaciones'], 
#               index=False, startrow = 2, header= False) 
       
    

         fecha     entrada  salida  hs_trab  observaciones      nombre
0                                                                     
1       Nombre  Pedro Meza                                            
2        fecha     entrada  salida  hs_trab  observaciones      nombre
3   2021-08-23               07:00     None              -  Pedro Meza
4   2021-08-24       07:00   13:02     6:01              -  Pedro Meza
5   2021-08-25               13:03     None              -  Pedro Meza
6   2021-08-26               13:03     None              -  Pedro Meza
7   2021-08-27               20:00     None              -  Pedro Meza
8   2021-08-30       14:06   20:02     5:56              -  Pedro Meza
9   2021-08-31       13:50   20:02     6:11              -  Pedro Meza
10  2021-09-01       13:54   20:01     6:07              -  Pedro Meza
11  2021-09-02               20:01     None              -  Pedro Meza
12  2021-09-03       13:54   20:00     6:05              -  Pedro Meza
13  20